### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [10]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio, MCPServerSse
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [2]:
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-memory"]}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create multiple new entities in the knowledge graph', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the entity'}, 'entityType': {'type': 'string', 'description': 'The type of the entity'}, 'observations': {'type': 'array', 'items': {'type': 'string'}, 'description': 'An array of observation contents associated with the entity'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='create_relations', description='Create multiple new relations between entities in the knowledge graph. Relations should be in active voice', inputSchema={'type': 'object', 'properties': {'relations': {'type': 'array', 'items': {'type': 'object', 'properties': {'from': {'type': 'string', 'description': 'The name of the entity where the relation starts'}, 'to': {'type': 'string', 'de

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = f"My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4o-mini"

In [4]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I've noted your details and the information about the MCP protocol. Here’s a summary of what I've saved:

- **Name:** Ed
  - **Role:** LLM Engineer
  - **Observations:** 
    - Ed is teaching a course about AI Agents.
    - Ed is knowledgeable about the MCP protocol.

- **MCP Protocol**
  - **Type:** Protocol
  - **Observations:**
    - It connects agents with tools, resources, and prompt templates.
    - It simplifies the integration of AI agents with various capabilities.

- **AI Agents**
  - **Type:** Course
  - **Observations:**
    - The course covers the MCP protocol and its application in AI agents.

If there's anything specific you'd like to add or modify, just let me know!

In [5]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

I have some information about you, Ed:

- You are an LLM engineer.
- You are teaching a course about AI Agents.
- You are knowledgeable about the MCP protocol. 

If there's anything more you'd like to add or update, feel free to let me know!

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [6]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

In [ ]:
mcp_tools

In [8]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

### And now the third time: running remotely

Note that as of May 4th 2025, this remote server has stopped responding - perhaps due to a problem with the MCP Server?
Please watch the video, and give this a try if you wish, otherwise skip to the next section with the STOP sign where we will configure a reliable MCP server!

This is the operational hazzard of working with remote MCP servers!

The remote server is:

https://smithery.ai/server/@qubaomingg/stock-analysis-mcp

First we need to set up a free Share Price account with AlphaVantage at:

https://www.alphavantage.co/

Click "Get Free API Key"

And enter that in your .env as `ALPHA_VANTAGE_API_KEY`

You also need to create a Smithery account at:

https://smithery.ai

And create a new free API key, and enter that in your .env as `SMITHERY_API_KEY`




In [11]:
import json
import base64

alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
smithery_api_key = os.getenv("SMITHERY_API_KEY")

config = {"alphaVantageApiKey": alpha_vantage_api_key}
config_b64 = base64.b64encode(json.dumps(config).encode()).decode()
url = f"wss://server.smithery.ai/@qubaomingg/stock-analysis-mcp/ws?config={config_b64}&api_key={smithery_api_key}"


#### Difficulties with the OpenAI Agents SDK SSE Client

This code should work (I believe) but it didn't!

```
params = {"url": url}
async with MCPServerSse(params=params) as server:
    mcp_tools = await server.list_tools()
```

So I wrote a quick client myself - see financial_datasets_client.py

In [13]:
# As of May 2025, this is hanging - hopefully it will start working again soon! Otherwise, please skip to the next section with the STOP sign..

from alpha_client import get_stock_tools_openai
openai_tools = await get_stock_tools_openai()

InvalidStatus: server rejected WebSocket connection: HTTP 500

In [14]:
openai_tools

NameError: name 'openai_tools' is not defined

In [13]:
instructions = "You can use tools to get stock prices."
request = f"Please let me know the share price of Amazon."
model = "gpt-4o-mini"

In [ ]:
agent = Agent(name="agent", instructions=instructions, model=model, tools=openai_tools)
with trace("conversation"):
    result = await Runner.run(agent, request)
display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

# New section - Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">The next section is NEW and added since the videos (and I will incorporate in the videos soon). I've discovered a much better service for financial market data that has both a FREE plan and a PAID plan, and we can use either depending on your appetite. This replaces the "Financial Datasets" MCP server, which at $0.01 for every single equity lookup, was not cost effective!
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [15]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [16]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=200.85, high=201.96, low=196.78, open=199.37, timestamp=1748635200000, volume=70819942.0, vwap=199.9641)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [17]:
from market import get_share_price
get_share_price("AAPL")

200.85

In [18]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

200.85

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [19]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [20]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4o-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $200.85.

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [21]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


[Tool(name='get_aggs', description='\n    List aggregate bars for a ticker over a given date range in custom time window sizes.\n    ', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'

### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [23]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4o-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I'm unable to retrieve the latest share price of Apple due to authorization restrictions. You can check the price on a financial news website or trading platform. If you have another inquiry or need different information, feel free to ask!

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>